In [14]:
# Exercise 01  —  Basic operations
# ----------------------------------------------------------
import pandas as pd
from pathlib import Path

DATA_DIR = Path("..") / "data"
VIEWS_FILE = DATA_DIR / "feed-views.log"

views = pd.read_csv(
    VIEWS_FILE,
    header=None,
    names=["datetime","user"],
    sep="\t",
    parse_dates=["datetime"],
    dtype={"user":"string"},
)


views["year"]   = views["datetime"].dt.year
views["month"]  = views["datetime"].dt.month
views["day"]    = views["datetime"].dt.day
views["hour"]   = views["datetime"].dt.hour
views["minute"] = views["datetime"].dt.minute
views["second"] = views["datetime"].dt.second

bins  = [0, 4, 7, 11, 17, 20, 24]
labels = [
    "night", "early morning", "morning",
    "afternoon", "early evening", "evening"
]
views["daytime"] = pd.cut(
    views["hour"],
    bins=bins,
    labels=labels,
    right=False,
    ordered=True
)

views.set_index("user", inplace=True)

total_rows = views.count().iloc[0]
by_daytime = views["daytime"].value_counts()

views_sorted = views.sort_values(
    ["hour", "minute", "second"]
)

hour_min  = views["hour"].min()
hour_max  = views["hour"].max()
hour_mode = views["hour"].mode()[0]

hour_stats = {
    "min":  hour_min,
    "max":  hour_max,
    "mode": hour_mode
}

night_max   = views.loc[views["daytime"] == "night", "hour"].max()
night_user  = views_sorted.query("daytime == 'night' & hour == @night_max").index[0]

morning_min = views.loc[views["daytime"] == "morning", "hour"].min()
morning_user= views_sorted.query("daytime == 'morning' & hour == @morning_min").index[0]

top3_early = views.nsmallest(3, ["hour", "minute", "second"])
top3_late  = views.nlargest(3, ["hour", "minute", "second"])

desc = views[["hour"]].describe()
iqr  = desc.loc["75%","hour"] - desc.loc["25%","hour"]



тесты

In [15]:
views.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1076 entries, artem to artem
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  1076 non-null   datetime64[ns]
 1   year      1076 non-null   int32         
 2   month     1076 non-null   int32         
 3   day       1076 non-null   int32         
 4   hour      1076 non-null   int32         
 5   minute    1076 non-null   int32         
 6   second    1076 non-null   int32         
 7   daytime   1076 non-null   category      
dtypes: category(1), datetime64[ns](1), int32(6)
memory usage: 75.6 KB


In [16]:
views.count()

datetime    1076
year        1076
month       1076
day         1076
hour        1076
minute      1076
second      1076
daytime     1076
dtype: int64

In [17]:
views.daytime.value_counts()

daytime
evening          509
afternoon        252
early evening    145
night            129
morning           36
early morning      5
Name: count, dtype: int64

In [18]:
views.loc[views.daytime == 'night'].hour.idxmax()

'konstantin'

In [19]:
views.loc[views.daytime == 'morning'].hour.idxmin()

'alexander'

In [20]:
views.hour.mode()

0    22
Name: hour, dtype: int32

In [21]:
views.daytime.mode()

0    evening
Name: daytime, dtype: category
Categories (6, object): ['night' < 'early morning' < 'morning' < 'afternoon' < 'early evening' < 'evening']

In [22]:
iqr

9.0